In [61]:
import pandas as pd

In [62]:
def search_in_price_list(price_list, phone_number):
    # search for a code match in a list
    #match = [s for s in price_list if (phone_number.startswith(s["code"]))]
    match = [price_list["code"][i] for i in range(price_list.shape[0]) if phone_number.startswith(price_list["code"][i])]
    #if the operator doesn't support the phone number
    if len(match) == 0:
        return 100000000000

    # search for the most exact code (longest)
    code = max(match, key=len)
    # search for the equivalent price
    price = price_list[price_list["code"] == code]["price"].iloc[0]
    return price

In [63]:
from operator import is_not
from functools import  partial

In [111]:
def selector(files, phone):
    n = len(files)
    prices = [None]*n
    for i in range(n):
        try:
            df = pd.read_csv(files[i], header = None)
            df.columns=["code","price"]
            df["code"] = df["code"].astype(str)
            prices[i]= search_in_price_list(df, phone)
        except FileNotFoundError:
            print("file not found")
    
    #all files doesn't exist
    if prices == [None]*n:
        return -1,-1
    
    min_price = min( filter(partial(is_not, None),prices ))
    if min_price == 100000000000 :
        print('this phone number is supported by no operator!')
        return -1, -1
    else:
        op = prices.index(min_price)
        print('best price: ', min_price, ' proposed by operator: ', files[op].replace('.csv', ''))
    
    return op, min_price

In [75]:
def __main__():
    phone = input("Enter the phone number: ")
    n = int(input("enter the number of operators: "))
    operators = [None]*n

    #prices = [None]*n

    #saving all price lists' file names
    for i in range(n):
        file_name =  input ("enter the csv price file's name: ")
        #save the operators price file name
        operators[i] = file_name

    selector(operators, phone)


In [66]:
#unit tests

In [67]:
import unittest

In [114]:
class TestSelector(unittest.TestCase):
    def test_search(self):
        df = pd.read_csv('file1.csv', header = None)
        df.columns=["code","price"]
        df["code"] = df["code"].astype(str)
        
        #the price of an existing code
        self.assertEqual(search_in_price_list(df, '4670193093'), 0.19)
        
        #the price of a code that doesn't exit
        self.assertEqual(search_in_price_list(df, '5677993093'), 100000000000)

    def test_selector(self):
        files = ['file1.csv','file2.csv','file3.csv']
        #if a code is supported by one operator
        self.assertEqual(selector(files, '14670193093'), (0, 0.9))
        
        #if code exists in supported by more than one operator
        self.assertEqual(selector(files, '4670193093'), (2, 0.17))
        
        #if a number is not supported by any operator 
        self.assertEqual(selector(files, '9874670193093'), (-1, -1))
        
        missing_file = ['file1.csv','file2.csv','file3.csv, randomfile']
        #a missing file
        self.assertEqual(selector(missing_file, '14670193093'), (0, 0.9))
        
        no_files = ['q','q','qd']
        #file not found
        self.assertEqual(selector(no_files, '14670193093'), (-1, -1))
        
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_search (__main__.TestSelector) ... ok
test_selector (__main__.TestSelector) ... 

best price:  0.9  proposed by operator:  file1
best price:  0.17  proposed by operator:  file3
this phone number is supported by no operator!
file not found
best price:  0.9  proposed by operator:  file1
file not found
file not found
file not found


ok

----------------------------------------------------------------------
Ran 2 tests in 0.069s

OK
